In [32]:
pip install opencv-python

Note: you may need to restart the kernel to use updated packages.


In [33]:
pip install pillow

Note: you may need to restart the kernel to use updated packages.


In [34]:
# -*- coding: utf-8 -*-
import os
import argparse
from PIL import Image
from PIL import ImageOps
from PIL import ImageFilter
from PIL import ImageEnhance
from cv2 import bilateralFilter
import numpy as np


def crop_image_uniform(src_dir, dst_dir):
    f = open("399-uniform.txt", "r")
    if not os.path.exists(dst_dir):
        os.makedirs(dst_dir)
    for page in range(1,4):
        img = Image.open( src_dir + "/" + str(page) +"-uniform.png").convert('L')

        width, height = img.size
        cell_width = width/float(cols)
        cell_height = height/float(rows)
        header_offset = height/float(rows) * header_ratio
        width_margin = cell_width * 0.10
        height_margin = cell_height * 0.10

        for j in range(0,rows):
            for i in range(0,cols):
                left = i * cell_width
                upper = j * cell_height + header_offset
                right = left + cell_width
                lower = (j+1) * cell_height

                center_x = (left + right) / 2
                center_y = (upper + lower) / 2

                crop_width = right - left - 2*width_margin
                crop_height = lower - upper - 2*height_margin

                size = 0
                if crop_width > crop_height:
                    size = crop_height/2
                else:
                    size = crop_width/2

                left = center_x - size;
                right = center_x + size;
                upper = center_y - size;
                lower = center_y + size;

                code = f.readline()
                if not code:
                    break
                else:
                    name = dst_dir + "/uni" + code.strip() + ".png"
                    cropped_image = img.crop((left, upper, right, lower))
                    cropped_image = cropped_image.resize((128,128), Image.LANCZOS)
                    # Increase constrast
                    enhancer = ImageEnhance.Contrast(cropped_image)
                    cropped_image = enhancer.enhance(1.5)
                    opencv_image = np.array(cropped_image)
                    opencv_image = bilateralFilter(opencv_image, 9, 30, 30)
                    cropped_image = Image.fromarray(opencv_image)
                    cropped_image.save(name)
        print("Processed uniform page " + str(page))
        

#parser = argparse.ArgumentParser(description='Crop scanned images to character images')
#parser.add_argument('--src_dir', dest='src_dir', required=True, help='directory to read scanned images')
#parser.add_argument('--dst_dir', dest='dst_dir', required=True, help='directory to save character images')

#args = parser.parse_args()




if __name__ == "__main__":
    rows = 12
    cols = 12
    header_ratio = 16.5/(16.5+42)
    crop_image_uniform('C:/Users/1-16/python/neural-fonts-master/src_dir', 'C:/Users/1-16/python/neural-fonts-master/dst_dir')
#    crop_image_frequency(args.src_dir, args.dst_dir)


Processed uniform page 1
Processed uniform page 2
Processed uniform page 3


In [35]:
# -*- coding: utf-8 -*-
from __future__ import print_function
from __future__ import absolute_import

import argparse
import sys
import numpy as np
import os
from PIL import Image
from PIL import ImageDraw
from PIL import ImageFont
import collections
from importlib import reload

reload(sys)
#sys.setdefaultencoding("utf-8")

KR_CHARSET = None

def get_offset(ch, font, canvas_size):
    font_size = font.getsize(ch)
    font_offset = font.getoffset(ch)
    offset_x = canvas_size/2 - font_size[0]/2 - font_offset[0]/2
    offset_y = canvas_size/2 - font_size[1]/2 - font_offset[1]/2
    return [ offset_x, offset_y ]


def draw_single_char(ch, font, canvas_size, x_offset, y_offset):
    img = Image.new("RGB", (canvas_size, canvas_size), (255, 255, 255)).convert('L')
    draw = ImageDraw.Draw(img)
    draw.text((x_offset, y_offset), ch, 0, font=font)
    return img


def draw_example(ch, src_font, dst_font, canvas_size, src_offset, dst_offset, filter_hashes):
    dst_img = draw_single_char(ch, dst_font, canvas_size, dst_offset[0], dst_offset[1])
    # check the filter example in the hashes or not
    dst_hash = hash(dst_img.tobytes())
    if dst_hash in filter_hashes:
        return None
    src_img = draw_single_char(ch, src_font, canvas_size, src_offset[0], src_offset[1])
    example_img = Image.new("RGB", (canvas_size * 2, canvas_size), (255, 255, 255)).convert('L')
    example_img.paste(dst_img, (0, 0))
    example_img.paste(src_img, (canvas_size, 0))
    return example_img


def get_font_offset(charset, font, canvas_size, filter_hashes):
    _charset = charset[:]
    np.random.shuffle(_charset)
    sample = _charset[:2000]
    font_offset = np.array([0, 0],dtype= np.float32)
    count = 0
    for c in sample:
        font_img = draw_single_char(c, font, canvas_size, 0, 0)
        font_hash = hash(font_img.tobytes())
        if not font_hash in filter_hashes:
            font_offset += get_offset(c, font, canvas_size)
            count += 1
    font_offset /= count
    return font_offset

def filter_recurring_hash(charset, font, canvas_size, x_offset, y_offset):
    """ Some characters are missing in a given font, filter them
    by checking the recurring hashes
    """
    _charset = charset[:]
    np.random.shuffle(_charset)
    sample = _charset[:2000]
    hash_count = collections.defaultdict(int)
    for c in sample:
        img = draw_single_char(c, font, canvas_size, x_offset, y_offset)
        hash_count[hash(img.tobytes())] += 1
    recurring_hashes = filter(lambda d: d[1] > 2, hash_count.items())
    return [rh[0] for rh in recurring_hashes]

def select_sample(charset):
    # this returns 399 samples from KR charset
    # we selected 399 characters to sample as uniformly as possible
    # (the number of each ChoSeong is fixed to 21 (i.e., 21 Giyeok, 21 Nieun ...))
    # Given the designs of these 399 characters, the rest of Hangeul will be generated
    samples = []
    for i in range(399):
        samples.append(charset[28*i+(i%28)])
    np.random.shuffle(samples)
    return samples


def draw_handwriting(ch, src_font, canvas_size, src_offset, dst_folder):
 
    s = ch.encode('raw_unicode_escape').decode().replace("\\u","").upper()
    dst_path = dst_folder + "/uni" + s + ".png"
    if not os.path.exists(dst_path):
        return
    dst_img = Image.open(dst_path)
    # check the filter example in the hashes or not
    src_img = draw_single_char(ch, src_font, canvas_size, src_offset[0], src_offset[1])
    example_img = Image.new("RGB", (canvas_size * 2, canvas_size), (255, 255, 255)).convert('L')
    example_img.paste(dst_img, (0, 0))
    example_img.paste(src_img, (canvas_size, 0))
    return example_img

def font2img(src, dst, charset, char_size, canvas_size,
             x_offset, y_offset, sample_count, sample_dir, label=0, filter_by_hash=True, fixed_sample=False, all_sample=False, handwriting_dir=False):
    src_font = ImageFont.truetype(src, size=char_size)
    dst_font = ImageFont.truetype(dst, size=char_size)

    dst_filter_hashes = set(filter_recurring_hash(charset, dst_font, canvas_size, 0, 0))
    dst_offset = get_font_offset(charset, dst_font, canvas_size, dst_filter_hashes)
   

    filter_hashes = set()
    if filter_by_hash:
        filter_hashes = set(filter_recurring_hash(charset, dst_font, canvas_size, dst_offset[0], dst_offset[1]))
        print("filter hashes -> %s" % (",".join([str(h) for h in filter_hashes])))

    count = 0

    if handwriting_dir:
        if not os.path.exists(sample_dir):
            os.makedirs(sample_dir)
        train_set = []
        for c in charset:
            e = draw_handwriting(c, src_font, canvas_size, [x_offset, y_offset], handwriting_dir)
            if e:
                e.save(os.path.join(sample_dir, "%d_%s_train.png" % (label, c.encode('raw_unicode_escape').decode().replace("\\u","").upper())))
                train_set.append(c)
                count += 1
                if count % 100 == 0:
                    print("processed %d chars" % count)

        np.random.shuffle(charset)
        count = 0
        for c in charset:
            e = draw_example(c, src_font, dst_font, canvas_size, [x_offset, y_offset], dst_offset, filter_hashes=set())
            if e:
                e.save(os.path.join(sample_dir, "%d_%s_val.png" % (label, c.encode('raw_unicode_escape').decode().replace("\\u","").upper())))
                count += 1
                if count % 100 == 0:
                    print("processed %d chars" % count)
        return

    if fixed_sample:
        train_set = select_sample(charset)
        for c in train_set:
            e = draw_example(c, src_font, dst_font, canvas_size, [x_offset, y_offset], dst_offset, filter_hashes)
            if e:
                e.save(os.path.join(sample_dir, "%d_%04d_train.png" % (label, count)))
                count += 1
                if count % 100 == 0:
                    print("processed %d chars" % count)

        np.random.shuffle(charset)
        count = 0
        for c in charset:
            if count == sample_count:
                break
            if c in train_set:
                continue
            e = draw_example(c, src_font, dst_font, canvas_size, [x_offset, y_offset], dst_offset, filter_hashes=set())
            if e:
                e.save(os.path.join(sample_dir, "%d_%04d_val.png" % (label, count)))
                count += 1
                if count % 100 == 0:
                    print("processed %d chars" % count)
        return

    if all_sample:
        for c in charset:
            e = draw_example(c, src_font, dst_font, canvas_size, [x_offset, y_offset], dst_offset, filter_hashes)
            if e:
                e.save(os.path.join(sample_dir, "%d_%04d.png" % (label, count)))
                count += 1
                if count % 1000 == 0:
                    print("processed %d chars" % count)
        return

    for c in charset:
        if count == sample_count:
            break
        e = draw_example(c, src_font, dst_font, canvas_size, [x_offset, y_offset], dst_offset, filter_hashes)
        if e:
            e.save(os.path.join(sample_dir, "%d_%04d.png" % (label, count)))
            count += 1
            if count % 100 == 0:
                print("processed %d chars" % count)



import easydict

args = easydict.EasyDict({
 
        "src_font": 'C:/Users/1-16/python/neural-fonts-master/fonts/NanumGothic.ttf',
 
        "dst_font": 'C:/Users/1-16/python/neural-fonts-master/fonts/NanumGothic.ttf',
 
        "filter": 0,
 
        #"shuffle": 0,
 
        "char_size": 80,
 
        "canvas_size": 128,
    
        "x_offset": 27,
        
        "y_offset": 16,
        
        "sample_count": 1000,
    
        "sample_dir": 'C:/Users/1-16/python/neural-fonts-master/sample_dir',
    
        "label": 0,
    
        "fixed_sample": 0,
    
        "all_sample": 0,
    
        "handwriting_dir": 'C:/Users/1-16/python/neural-fonts-master/dst_dir',

 
})


if __name__ == "__main__":
    charset = []
    for i in range(0xac00,0xd7a4):
        charset.append(chr(i))
    for i in range(0x3131,0x3164):
        charset.append(chr(i))
    #if args.shuffle:
    np.random.shuffle(charset)
    font2img(args.src_font, args.dst_font, charset, args.char_size,
             args.canvas_size, args.x_offset, args.y_offset,
             args.sample_count, args.sample_dir, args.label, args.filter, args.fixed_sample, args.all_sample, args.handwriting_dir)


processed 100 chars
processed 200 chars
processed 300 chars
processed 100 chars
processed 200 chars
processed 300 chars
processed 400 chars
processed 500 chars
processed 600 chars
processed 700 chars
processed 800 chars
processed 900 chars
processed 1000 chars
processed 1100 chars
processed 1200 chars
processed 1300 chars
processed 1400 chars
processed 1500 chars
processed 1600 chars
processed 1700 chars
processed 1800 chars
processed 1900 chars
processed 2000 chars
processed 2100 chars
processed 2200 chars
processed 2300 chars
processed 2400 chars
processed 2500 chars
processed 2600 chars
processed 2700 chars
processed 2800 chars
processed 2900 chars
processed 3000 chars
processed 3100 chars
processed 3200 chars
processed 3300 chars
processed 3400 chars
processed 3500 chars
processed 3600 chars
processed 3700 chars
processed 3800 chars
processed 3900 chars
processed 4000 chars
processed 4100 chars
processed 4200 chars
processed 4300 chars
processed 4400 chars
processed 4500 chars
proc

In [36]:
pip install easydict

Note: you may need to restart the kernel to use updated packages.


In [37]:
# -*- coding: utf-8 -*-
from __future__ import print_function
from __future__ import absolute_import

import argparse
import glob
import os
import pickle 
import random


def pickle_examples(paths, train_path, val_path, train_val_split=0.1, fixed_sample=False):
    """
    Compile a list of examples into pickled format, so during
    the training, all io will happen in memory
    """
    if fixed_sample:
        with open(train_path, 'wb') as ft:
            with open(val_path, 'wb') as fv:
                for p in paths:
                    label = int(os.path.basename(p).split("_")[0])
                    uni = os.path.basename(p).split("_")[1]
                    with open(p, 'rb') as f:
#                        print("img %s" % p, label)
                        img_bytes = f.read()
                        example = (label, uni, img_bytes)
                        if "val" in p:
#                            print("img %s is saved in val.obj" % p)
                            # validation set
                            pickle.dump(example, fv)
                        else:
                            # training set
#                            print("img %s is saved in train.obj" % p)
                            pickle.dump(example, ft)
                return
    with open(train_path, 'wb') as ft:
        with open(val_path, 'wb') as fv:
            for p in paths:
                label = int(os.path.basename(p).split("_")[0])
                with open(p, 'rb') as f:
                    print("img %s" % p, label)
                    img_bytes = f.read()
                    r = random.random()
                    example = (label, img_bytes)
                    if r < train_val_split:
                        pickle.dump(example, fv)
                    else:
                        pickle.dump(example, ft)


# parser = argparse.ArgumentParser(description='Compile list of images into a pickled object for training')
# parser.add_argument('--dir', dest='dir', required=True, help='path of examples')
# parser.add_argument('--save_dir', dest='save_dir', required=True, help='path to save pickled files')
# parser.add_argument('--split_ratio', type=float, default=0.1, dest='split_ratio',
#                     help='split ratio between train and val')
# parser.add_argument('--fixed_sample', dest='fixed_sample', default=0, help='binarize fixed samples (we distiguish train/validation data with its file name).')
# args = parser.parse_args()

import easydict

args = easydict.EasyDict({
 
        "dir": 'C:/Users/1-16/python/neural-fonts-master/sample_dir',
 
        "save_dir": 'C:/Users/1-16/python/neural-fonts-master/save_dir',

 
        "split_ratio": 0.1,
 
        "fixed_sample": 0,
 
    
 
})


if __name__ == "__main__":
    if not os.path.exists(args.save_dir):
        os.makedirs(args.save_dir)
    train_path = os.path.join(args.save_dir, "train.obj")
    val_path = os.path.join(args.save_dir, "val.obj")
    pickle_examples(glob.glob(os.path.join(args.dir, "*.png")), train_path=train_path, val_path=val_path,
                    train_val_split=args.split_ratio, fixed_sample=args.fixed_sample)
#    pickle_examples(sorted(glob.glob(os.path.join(args.dir, "*.png")), key=lambda e: float(os.path.splitext(os.path.basename(e))[0].replace("_","").replace("train","").replace("val",""))), train_path=train_path, val_path=val_path,
#                    train_val_split=args.split_ratio, fixed_sample=args.fixed_sample)


img C:/Users/1-16/python/neural-fonts-master/sample_dir\0_3131_val.png 0
img C:/Users/1-16/python/neural-fonts-master/sample_dir\0_3132_val.png 0
img C:/Users/1-16/python/neural-fonts-master/sample_dir\0_3133_val.png 0
img C:/Users/1-16/python/neural-fonts-master/sample_dir\0_3134_val.png 0
img C:/Users/1-16/python/neural-fonts-master/sample_dir\0_3135_val.png 0
img C:/Users/1-16/python/neural-fonts-master/sample_dir\0_3136_val.png 0
img C:/Users/1-16/python/neural-fonts-master/sample_dir\0_3137_val.png 0
img C:/Users/1-16/python/neural-fonts-master/sample_dir\0_3138_val.png 0
img C:/Users/1-16/python/neural-fonts-master/sample_dir\0_3139_val.png 0
img C:/Users/1-16/python/neural-fonts-master/sample_dir\0_313A_val.png 0
img C:/Users/1-16/python/neural-fonts-master/sample_dir\0_313B_val.png 0
img C:/Users/1-16/python/neural-fonts-master/sample_dir\0_313C_val.png 0
img C:/Users/1-16/python/neural-fonts-master/sample_dir\0_313D_val.png 0
img C:/Users/1-16/python/neural-fonts-master/sample

img C:/Users/1-16/python/neural-fonts-master/sample_dir\0_ACE7_val.png 0
img C:/Users/1-16/python/neural-fonts-master/sample_dir\0_ACE8_train.png 0
img C:/Users/1-16/python/neural-fonts-master/sample_dir\0_ACE8_val.png 0
img C:/Users/1-16/python/neural-fonts-master/sample_dir\0_ACE9_val.png 0
img C:/Users/1-16/python/neural-fonts-master/sample_dir\0_ACEA_val.png 0
img C:/Users/1-16/python/neural-fonts-master/sample_dir\0_ACEB_val.png 0
img C:/Users/1-16/python/neural-fonts-master/sample_dir\0_ACEC_val.png 0
img C:/Users/1-16/python/neural-fonts-master/sample_dir\0_ACED_val.png 0
img C:/Users/1-16/python/neural-fonts-master/sample_dir\0_ACEE_val.png 0
img C:/Users/1-16/python/neural-fonts-master/sample_dir\0_ACEF_val.png 0
img C:/Users/1-16/python/neural-fonts-master/sample_dir\0_ACF0_val.png 0
img C:/Users/1-16/python/neural-fonts-master/sample_dir\0_ACF1_val.png 0
img C:/Users/1-16/python/neural-fonts-master/sample_dir\0_ACF2_val.png 0
img C:/Users/1-16/python/neural-fonts-master/samp

img C:/Users/1-16/python/neural-fonts-master/sample_dir\0_ADAF_val.png 0
img C:/Users/1-16/python/neural-fonts-master/sample_dir\0_ADB0_val.png 0
img C:/Users/1-16/python/neural-fonts-master/sample_dir\0_ADB1_val.png 0
img C:/Users/1-16/python/neural-fonts-master/sample_dir\0_ADB2_val.png 0
img C:/Users/1-16/python/neural-fonts-master/sample_dir\0_ADB3_train.png 0
img C:/Users/1-16/python/neural-fonts-master/sample_dir\0_ADB3_val.png 0
img C:/Users/1-16/python/neural-fonts-master/sample_dir\0_ADB4_val.png 0
img C:/Users/1-16/python/neural-fonts-master/sample_dir\0_ADB5_val.png 0
img C:/Users/1-16/python/neural-fonts-master/sample_dir\0_ADB6_val.png 0
img C:/Users/1-16/python/neural-fonts-master/sample_dir\0_ADB7_val.png 0
img C:/Users/1-16/python/neural-fonts-master/sample_dir\0_ADB8_val.png 0
img C:/Users/1-16/python/neural-fonts-master/sample_dir\0_ADB9_val.png 0
img C:/Users/1-16/python/neural-fonts-master/sample_dir\0_ADBA_val.png 0
img C:/Users/1-16/python/neural-fonts-master/samp

img C:/Users/1-16/python/neural-fonts-master/sample_dir\0_AED7_val.png 0
img C:/Users/1-16/python/neural-fonts-master/sample_dir\0_AED8_val.png 0
img C:/Users/1-16/python/neural-fonts-master/sample_dir\0_AED9_val.png 0
img C:/Users/1-16/python/neural-fonts-master/sample_dir\0_AEDA_val.png 0
img C:/Users/1-16/python/neural-fonts-master/sample_dir\0_AEDB_val.png 0
img C:/Users/1-16/python/neural-fonts-master/sample_dir\0_AEDC_val.png 0
img C:/Users/1-16/python/neural-fonts-master/sample_dir\0_AEDD_val.png 0
img C:/Users/1-16/python/neural-fonts-master/sample_dir\0_AEDE_val.png 0
img C:/Users/1-16/python/neural-fonts-master/sample_dir\0_AEDF_val.png 0
img C:/Users/1-16/python/neural-fonts-master/sample_dir\0_AEE0_val.png 0
img C:/Users/1-16/python/neural-fonts-master/sample_dir\0_AEE1_val.png 0
img C:/Users/1-16/python/neural-fonts-master/sample_dir\0_AEE2_val.png 0
img C:/Users/1-16/python/neural-fonts-master/sample_dir\0_AEE3_val.png 0
img C:/Users/1-16/python/neural-fonts-master/sample

img C:/Users/1-16/python/neural-fonts-master/sample_dir\0_AFAD_val.png 0
img C:/Users/1-16/python/neural-fonts-master/sample_dir\0_AFAE_val.png 0
img C:/Users/1-16/python/neural-fonts-master/sample_dir\0_AFAF_val.png 0
img C:/Users/1-16/python/neural-fonts-master/sample_dir\0_AFB0_val.png 0
img C:/Users/1-16/python/neural-fonts-master/sample_dir\0_AFB1_val.png 0
img C:/Users/1-16/python/neural-fonts-master/sample_dir\0_AFB2_val.png 0
img C:/Users/1-16/python/neural-fonts-master/sample_dir\0_AFB3_val.png 0
img C:/Users/1-16/python/neural-fonts-master/sample_dir\0_AFB4_val.png 0
img C:/Users/1-16/python/neural-fonts-master/sample_dir\0_AFB5_val.png 0
img C:/Users/1-16/python/neural-fonts-master/sample_dir\0_AFB6_val.png 0
img C:/Users/1-16/python/neural-fonts-master/sample_dir\0_AFB7_val.png 0
img C:/Users/1-16/python/neural-fonts-master/sample_dir\0_AFB8_val.png 0
img C:/Users/1-16/python/neural-fonts-master/sample_dir\0_AFB9_val.png 0
img C:/Users/1-16/python/neural-fonts-master/sample

...

img C:/Users/1-16/python/neural-fonts-master/sample_dir\0_D3F8_val.png 0
img C:/Users/1-16/python/neural-fonts-master/sample_dir\0_D3F9_val.png 0
img C:/Users/1-16/python/neural-fonts-master/sample_dir\0_D3FA_val.png 0
img C:/Users/1-16/python/neural-fonts-master/sample_dir\0_D3FB_val.png 0
img C:/Users/1-16/python/neural-fonts-master/sample_dir\0_D3FC_val.png 0
img C:/Users/1-16/python/neural-fonts-master/sample_dir\0_D3FD_val.png 0
img C:/Users/1-16/python/neural-fonts-master/sample_dir\0_D3FE_val.png 0
img C:/Users/1-16/python/neural-fonts-master/sample_dir\0_D3FF_val.png 0
img C:/Users/1-16/python/neural-fonts-master/sample_dir\0_D400_val.png 0
img C:/Users/1-16/python/neural-fonts-master/sample_dir\0_D401_val.png 0
img C:/Users/1-16/python/neural-fonts-master/sample_dir\0_D402_val.png 0
img C:/Users/1-16/python/neural-fonts-master/sample_dir\0_D403_val.png 0
img C:/Users/1-16/python/neural-fonts-master/sample_dir\0_D404_val.png 0
img C:/Users/1-16/python/neural-fonts-master/sample

img C:/Users/1-16/python/neural-fonts-master/sample_dir\0_D4EF_val.png 0
img C:/Users/1-16/python/neural-fonts-master/sample_dir\0_D4F0_val.png 0
img C:/Users/1-16/python/neural-fonts-master/sample_dir\0_D4F1_val.png 0
img C:/Users/1-16/python/neural-fonts-master/sample_dir\0_D4F2_train.png 0
img C:/Users/1-16/python/neural-fonts-master/sample_dir\0_D4F2_val.png 0
img C:/Users/1-16/python/neural-fonts-master/sample_dir\0_D4F3_val.png 0
img C:/Users/1-16/python/neural-fonts-master/sample_dir\0_D4F4_val.png 0
img C:/Users/1-16/python/neural-fonts-master/sample_dir\0_D4F5_val.png 0
img C:/Users/1-16/python/neural-fonts-master/sample_dir\0_D4F6_val.png 0
img C:/Users/1-16/python/neural-fonts-master/sample_dir\0_D4F7_val.png 0
img C:/Users/1-16/python/neural-fonts-master/sample_dir\0_D4F8_val.png 0
img C:/Users/1-16/python/neural-fonts-master/sample_dir\0_D4F9_val.png 0
img C:/Users/1-16/python/neural-fonts-master/sample_dir\0_D4FA_val.png 0
img C:/Users/1-16/python/neural-fonts-master/samp

img C:/Users/1-16/python/neural-fonts-master/sample_dir\0_D580_val.png 0
img C:/Users/1-16/python/neural-fonts-master/sample_dir\0_D581_val.png 0
img C:/Users/1-16/python/neural-fonts-master/sample_dir\0_D582_val.png 0
img C:/Users/1-16/python/neural-fonts-master/sample_dir\0_D583_train.png 0
img C:/Users/1-16/python/neural-fonts-master/sample_dir\0_D583_val.png 0
img C:/Users/1-16/python/neural-fonts-master/sample_dir\0_D584_val.png 0
img C:/Users/1-16/python/neural-fonts-master/sample_dir\0_D585_val.png 0
img C:/Users/1-16/python/neural-fonts-master/sample_dir\0_D586_val.png 0
img C:/Users/1-16/python/neural-fonts-master/sample_dir\0_D587_val.png 0
img C:/Users/1-16/python/neural-fonts-master/sample_dir\0_D588_val.png 0
img C:/Users/1-16/python/neural-fonts-master/sample_dir\0_D589_val.png 0
img C:/Users/1-16/python/neural-fonts-master/sample_dir\0_D58A_val.png 0
img C:/Users/1-16/python/neural-fonts-master/sample_dir\0_D58B_val.png 0
img C:/Users/1-16/python/neural-fonts-master/samp

img C:/Users/1-16/python/neural-fonts-master/sample_dir\0_D6D2_val.png 0
img C:/Users/1-16/python/neural-fonts-master/sample_dir\0_D6D3_val.png 0
img C:/Users/1-16/python/neural-fonts-master/sample_dir\0_D6D4_val.png 0
img C:/Users/1-16/python/neural-fonts-master/sample_dir\0_D6D5_val.png 0
img C:/Users/1-16/python/neural-fonts-master/sample_dir\0_D6D6_val.png 0
img C:/Users/1-16/python/neural-fonts-master/sample_dir\0_D6D7_val.png 0
img C:/Users/1-16/python/neural-fonts-master/sample_dir\0_D6D8_val.png 0
img C:/Users/1-16/python/neural-fonts-master/sample_dir\0_D6D9_val.png 0
img C:/Users/1-16/python/neural-fonts-master/sample_dir\0_D6DA_val.png 0
img C:/Users/1-16/python/neural-fonts-master/sample_dir\0_D6DB_val.png 0
img C:/Users/1-16/python/neural-fonts-master/sample_dir\0_D6DC_val.png 0
img C:/Users/1-16/python/neural-fonts-master/sample_dir\0_D6DD_val.png 0
img C:/Users/1-16/python/neural-fonts-master/sample_dir\0_D6DE_val.png 0
img C:/Users/1-16/python/neural-fonts-master/sample

img C:/Users/1-16/python/neural-fonts-master/sample_dir\0_D772_val.png 0
img C:/Users/1-16/python/neural-fonts-master/sample_dir\0_D773_val.png 0
img C:/Users/1-16/python/neural-fonts-master/sample_dir\0_D774_val.png 0
img C:/Users/1-16/python/neural-fonts-master/sample_dir\0_D775_val.png 0
img C:/Users/1-16/python/neural-fonts-master/sample_dir\0_D776_val.png 0
img C:/Users/1-16/python/neural-fonts-master/sample_dir\0_D777_val.png 0
img C:/Users/1-16/python/neural-fonts-master/sample_dir\0_D778_val.png 0
img C:/Users/1-16/python/neural-fonts-master/sample_dir\0_D779_val.png 0
img C:/Users/1-16/python/neural-fonts-master/sample_dir\0_D77A_val.png 0
img C:/Users/1-16/python/neural-fonts-master/sample_dir\0_D77B_val.png 0
img C:/Users/1-16/python/neural-fonts-master/sample_dir\0_D77C_val.png 0
img C:/Users/1-16/python/neural-fonts-master/sample_dir\0_D77D_val.png 0
img C:/Users/1-16/python/neural-fonts-master/sample_dir\0_D77E_val.png 0
img C:/Users/1-16/python/neural-fonts-master/sample

In [93]:
# util.py
# -*- coding: utf-8 -*-
from __future__ import print_function
from __future__ import absolute_import

import os
import glob

import imageio
import scipy.misc as misc
import numpy as np
# from io import StringIO
import io

def pad_seq(seq, batch_size):
    # pad the sequence to be the multiples of batch_size
    seq_len = len(seq)
    if seq_len % batch_size == 0:
        return seq
    padded = batch_size - (seq_len % batch_size)
    seq.extend(seq[:padded])
    return seq


def bytes_to_file(bytes_img):
    return io.StringIO(bytes_img)


def normalize_image(img):
    """
    Make image zero centered and in between (-1, 1)
    """
    normalized = (img / 127.5) - 1.
    return normalized


def read_split_image(img):
    mat = misc.imread(img).astype(np.float)
    side = int(mat.shape[1] / 2)
    assert side * 2 == mat.shape[1]
    img_A = mat[:, :side]  # target
    img_B = mat[:, side:]  # source

    return img_A, img_B


def shift_and_resize_image(img, shift_x, shift_y, nw, nh):
#    w, h, _ = img.shape
    w, h = img.shape
    enlarged = misc.imresize(img, [nw, nh])
    return enlarged[shift_x:shift_x + w, shift_y:shift_y + h]


def scale_back(images):
    return (images + 1.) / 2.


def merge(images, size):
    h, w = images.shape[1], images.shape[2]
    img = np.zeros((h * size[0], w * size[1], 3))
    for idx, image in enumerate(images):
        i = idx % size[1]
        j = idx // size[1]
        img[j * h:j * h + h, i * w:i * w + w, :] = image

    return img


def save_concat_images(imgs, img_path):
    concated = np.concatenate(imgs, axis=1)
    misc.imsave(img_path, concated)


def compile_frames_to_gif(frame_dir, gif_file):
    frames = sorted(glob.glob(os.path.join(frame_dir, "*.png")))
    print(frames)
    images = [misc.imresize(imageio.imread(f), interp='nearest', size=0.33) for f in frames]
    imageio.mimsave(gif_file, images, duration=0.1)
    return gif_file

In [117]:
# dataset.py
# -*- coding: utf-8 -*-
from __future__ import print_function
from __future__ import absolute_import
import pickle 
import numpy as np
import random
import os
#from .utils import pad_seq, bytes_to_file, read_split_image, shift_and_resize_image, normalize_image


class PickledImageProvider(object):
    def __init__(self, obj_path):
        self.obj_path = obj_path
        self.examples = self.load_pickled_examples()

    def load_pickled_examples(self):
        with open(self.obj_path, "rb") as of:
            examples = list()
            while True:
                try:
                    e = pickle.load(of)
                    examples.append(e)
                    if len(examples) % 1000 == 0:
                        print("processed %d examples" % len(examples))
                except EOFError:
                    break
                except Exception:
                    pass
            print("unpickled total %d examples" % len(examples))
            return examples


def get_batch_iter(examples, batch_size, augment):
    # the transpose ops requires deterministic
    # batch size, thus comes the padding
    padded = pad_seq(examples, batch_size)
    print("padded :", padded)

    def process(img):
        img = bytes_to_file(img)
        try:
            img_A, img_B = read_split_image(img)
            if augment:
                # augment the image by:
                # 1) enlarge the image
                # 2) random crop the image back to its original size
                # NOTE: image A and B needs to be in sync as how much
                # to be shifted
#                w, h, _ = img_A.shape
                w, h = img_A.shape
                multiplier = random.uniform(1.00, 1.20)
                # add an eps to prevent cropping issue
                nw = int(multiplier * w) + 1
                nh = int(multiplier * h) + 1
                shift_x = int(np.ceil(np.random.uniform(0.01, nw - w)))
                shift_y = int(np.ceil(np.random.uniform(0.01, nh - h)))
                img_A = shift_and_resize_image(img_A, shift_x, shift_y, nw, nh)
                img_B = shift_and_resize_image(img_B, shift_x, shift_y, nw, nh)
            img_A = normalize_image(img_A).reshape((128,128,1))
            img_B = normalize_image(img_B).reshape((128,128,1))
            return np.concatenate((img_A, img_B), axis=2)
        finally:
            img.close()

    def batch_iter():
        for i in range(0, len(padded), batch_size):
            batch = padded[i: i + batch_size]
            # batch = padded[i:batch_size]
            print('batch len: ' , len(batch))
            labels = [e[0] for e in batch]
            print('labels len: ' , len(labels))
            codes = [e[1] for e in batch]
            print('codes len: ', len(codes))        
            processed = [process(e[2]) for e in batch]
           
            # stack into tensor
            yield labels, codes, np.array(processed).astype(np.float32)



    return batch_iter()


class TrainDataProvider(object):
    def __init__(self, data_dir, train_name="train.obj", val_name="val.obj", filter_by=None, no_val=False):
        self.data_dir = data_dir
        self.filter_by = filter_by
        self.train_path = os.path.join(self.data_dir, train_name)
        self.val_path = os.path.join(self.data_dir, val_name)
        self.train = PickledImageProvider(self.train_path)
        if not no_val:
            self.val = PickledImageProvider(self.val_path)
        if self.filter_by:
            print("filter by label ->", filter_by)
            self.train.examples = filter(lambda e: e[0] in self.filter_by, self.train.examples)
            if not no_val:
                self.val.examples = filter(lambda e: e[0] in self.filter_by, self.val.examples)
        if not no_val:
            print("train examples -> %d, val examples -> %d" % (len(self.train.examples), len(self.val.examples)))
        else:
            print("train examples -> %d" % (len(self.train.examples)))

    def get_train_iter(self, batch_size, shuffle=True):
        training_examples = self.train.examples[:]
        if shuffle:
            np.random.shuffle(training_examples)
        return get_batch_iter(training_examples, batch_size, augment=True)

    def get_val_iter(self, batch_size, shuffle=True):
        """
        Validation iterator runs forever
        """
        val_examples = self.val.examples[:]
        if shuffle:
            np.random.shuffle(val_examples)
        while True:
            val_batch_iter = get_batch_iter(val_examples, batch_size, augment=False)
            for labels, codes, examples in val_batch_iter:
                yield labels, codes, examples

    def compute_total_batch_num(self, batch_size):
        """Total padded batch num"""
        return int(np.ceil(len(self.train.examples) / float(batch_size)))

    def get_all_labels(self):
        """Get all training labels"""
        return list({e[0] for e in self.train.examples})

    def get_train_val_path(self):
        return self.train_path, self.val_path


class InjectDataProvider(object):
    def __init__(self, obj_path, filter_by=None):
        self.filter_by = filter_by
        self.data = PickledImageProvider(obj_path)
        if self.filter_by:
            print("filter by label ->", filter_by)
            self.data.examples = filter(lambda e: e[0] in self.filter_by, self.data.examples)
        print("examples -> %d" % len(self.data.examples))

    def get_single_embedding_iter(self, batch_size, embedding_id):
        examples = self.data.examples[:]
        batch_iter = get_batch_iter(examples, batch_size, augment=False)
        for _, codes, images in batch_iter:
            # inject specific embedding style here
            labels = [embedding_id] * batch_size
            yield labels, codes, images

    def get_random_embedding_iter(self, batch_size, embedding_ids):
        examples = self.data.examples[:]
        batch_iter = get_batch_iter(examples, batch_size, augment=False)
        for _, codes, images in batch_iter:
            # inject specific embedding style here
            labels = [random.choice(embedding_ids) for i in range(batch_size)]
            yield labels, codes, images


class NeverEndingLoopingProvider(InjectDataProvider):
    def __init__(self, obj_path):
        super(NeverEndingLoopingProvider, self).__init__(obj_path)

    def get_random_embedding_iter(self, batch_size, embedding_ids):
        while True:
            # np.random.shuffle(self.data.examples)
            rand_iter = super(NeverEndingLoopingProvider, self) \
                .get_random_embedding_iter(batch_size, embedding_ids)
            for labels, codes, images in rand_iter:
                yield labels, codes, images


In [95]:
# ops.py
# -*- coding: utf-8 -*-
from __future__ import print_function
from __future__ import absolute_import
import tensorflow as tf


def batch_normalization(x, is_training, epsilon=1e-5, decay=0.9, scope="batch_normalization"):
    return tf.layers.batch_normalization(x, momentum=decay, renorm=None, epsilon=epsilon,
                                        scale=True, training=is_training, name=scope)


def conv2d(x, output_filters, kh=5, kw=5, sh=2, sw=2, stddev=0.02, scope="conv2d"):
    with tf.variable_scope(scope):
        shape = x.get_shape().as_list()
        W = tf.get_variable('W', [kh, kw, shape[-1], output_filters],
                            initializer=tf.truncated_normal_initializer(stddev=stddev))
        Wconv = tf.nn.conv2d(x, W, strides=[1, sh, sw, 1], padding='SAME')

        biases = tf.get_variable('b', [output_filters], initializer=tf.constant_initializer(0.0))
        Wconv_plus_b = tf.reshape(tf.nn.bias_add(Wconv, biases), Wconv.get_shape())

        return Wconv_plus_b


def deconv2d(x, output_shape, kh=5, kw=5, sh=2, sw=2, stddev=0.02, scope="deconv2d"):
    with tf.variable_scope(scope):
        # filter : [height, width, output_channels, in_channels]
        input_shape = x.get_shape().as_list()
        W = tf.get_variable('W', [kh, kw, output_shape[-1], input_shape[-1]],
                            initializer=tf.random_normal_initializer(stddev=stddev))

        deconv = tf.nn.conv2d_transpose(x, W, output_shape=output_shape,
                                        strides=[1, sh, sw, 1])

        biases = tf.get_variable('b', [output_shape[-1]], initializer=tf.constant_initializer(0.0))
        deconv_plus_b = tf.reshape(tf.nn.bias_add(deconv, biases), deconv.get_shape())

        return deconv_plus_b


def lrelu(x, leak=0.2):
    return tf.maximum(x, leak * x)


def fc(x, output_size, stddev=0.02, scope="fc"):
    with tf.variable_scope(scope):
        shape = x.get_shape().as_list()
        W = tf.get_variable("W", [shape[1], output_size], tf.float32,
                            tf.random_normal_initializer(stddev=stddev))
        b = tf.get_variable("b", [output_size],
                            initializer=tf.constant_initializer(0.0))
        return tf.matmul(x, W) + b


def init_embedding(size, dimension, stddev=0.01, scope="embedding"):
    with tf.variable_scope(scope):
        return tf.get_variable("E", [size, 1, 1, dimension], tf.float32,
                               tf.random_normal_initializer(stddev=stddev))


def conditional_instance_norm(x, ids, labels_num, mixed=False, scope="conditional_instance_norm"):
    with tf.variable_scope(scope):
        shape = x.get_shape().as_list()
        batch_size, output_filters = shape[0], shape[-1]
        scale = tf.get_variable("scale", [labels_num, output_filters], tf.float32, tf.constant_initializer(1.0))
        shift = tf.get_variable("shift", [labels_num, output_filters], tf.float32, tf.constant_initializer(0.0))

        mu, sigma = tf.nn.moments(x, [1, 2], keep_dims=True)
        norm = (x - mu) / tf.sqrt(sigma + 1e-5)

        batch_scale = tf.reshape(tf.nn.embedding_lookup([scale], ids=ids), [batch_size, 1, 1, output_filters])
        batch_shift = tf.reshape(tf.nn.embedding_lookup([shift], ids=ids), [batch_size, 1, 1, output_filters])

        z = norm * batch_scale + batch_shift
        return z


In [111]:
# unet.py
# -*- coding: utf-8 -*-
from __future__ import print_function
from __future__ import absolute_import

import tensorflow as tf
import numpy as np
import scipy.misc as misc
import os
import time
from collections import namedtuple
#from .ops import conv2d, deconv2d, lrelu, fc, batch_norm, init_embedding, conditional_instance_norm
#from .dataset import TrainDataProvider, InjectDataProvider, NeverEndingLoopingProvider
#from .utils import scale_back, merge, save_concat_images
from skimage.measure import compare_ssim as ssim
from scipy import ndimage
from PIL import Image
from PIL import ImageEnhance
from cv2 import bilateralFilter

# Auxiliary wrapper classes
# Used to save handles(important nodes in computation graph) for later evaluation
LossHandle = namedtuple("LossHandle", ["d_loss", "g_loss", "const_loss", "l1_loss",
                                       "category_loss", "cheat_loss", "tv_loss"])
InputHandle = namedtuple("InputHandle", ["real_data", "embedding_ids", "no_target_data", "no_target_ids"])
EvalHandle = namedtuple("EvalHandle", ["encoder", "generator", "target", "source", "embedding"])
SummaryHandle = namedtuple("SummaryHandle", ["d_merged", "g_merged"])


class UNet(object):
    def __init__(self, experiment_dir=None, experiment_id=0, batch_size=16, input_width=128, output_width=128,
                 generator_dim=64, discriminator_dim=32, L1_penalty=100, Lconst_penalty=15, Ltv_penalty=0.0,
                 Lcategory_penalty=1.0, embedding_num=40, embedding_dim=128, input_filters=1, output_filters=1):
        self.experiment_dir = experiment_dir
        self.experiment_id = experiment_id
        self.batch_size = batch_size
        self.input_width = input_width
        self.output_width = output_width
        self.generator_dim = generator_dim
        self.discriminator_dim = discriminator_dim
        self.L1_penalty = L1_penalty
        self.Lconst_penalty = Lconst_penalty
        self.Ltv_penalty = Ltv_penalty
        self.Lcategory_penalty = Lcategory_penalty
        self.embedding_num = embedding_num
        self.embedding_dim = embedding_dim
        self.input_filters = input_filters
        self.output_filters = output_filters
        # init all the directories
        self.sess = None
        # experiment_dir is needed for training
        if experiment_dir:
            self.data_dir = os.path.join(self.experiment_dir, "data")
            self.checkpoint_dir = os.path.join(self.experiment_dir, "checkpoint")
            self.sample_dir = os.path.join(self.experiment_dir, "sample")
            self.log_dir = os.path.join(self.experiment_dir, "logs")
            self.progress_file = os.path.join(self.log_dir, "progress")

            if not os.path.exists(self.checkpoint_dir):
                os.makedirs(self.checkpoint_dir)
                print("create checkpoint directory")
            if not os.path.exists(self.log_dir):
                os.makedirs(self.log_dir)
                print("create log directory")
            if not os.path.exists(self.sample_dir):
                os.makedirs(self.sample_dir)
                print("create sample directory")

    def encoder(self, images, is_training, reuse=False):
        with tf.variable_scope("generator"):
            if reuse:
                tf.get_variable_scope().reuse_variables()

            encode_layers = dict()

            def encode_layer(x, output_filters, layer):
                act = lrelu(x)
                conv = conv2d(act, output_filters=output_filters, scope="g_e%d_conv" % layer)
                enc = batch_normalization(conv, is_training, scope="g_e%d_bn" % layer)
                encode_layers["e%d" % layer] = enc
                return enc

            e1 = conv2d(images, self.generator_dim, scope="g_e1_conv")
            encode_layers["e1"] = e1
            e2 = encode_layer(e1, self.generator_dim * 2, 2)
            e3 = encode_layer(e2, self.generator_dim * 4, 3)
            e4 = encode_layer(e3, self.generator_dim * 8, 4)
            e5 = encode_layer(e4, self.generator_dim * 8, 5)
            e6 = encode_layer(e5, self.generator_dim * 8, 6)
            e7 = encode_layer(e6, self.generator_dim * 8, 7)
            e8 = encode_layer(e7, self.generator_dim * 8, 8)

            return e8, encode_layers

    def decoder(self, encoded, encoding_layers, ids, inst_norm, is_training, reuse=False):
        with tf.variable_scope("generator"):
            if reuse:
                tf.get_variable_scope().reuse_variables()

            s = self.output_width
            s2, s4, s8, s16, s32, s64, s128 = int(s / 2), int(s / 4), int(s / 8), int(s / 16), int(s / 32), int(
                s / 64), int(s / 128)

            def decode_layer(x, output_width, output_filters, layer, enc_layer, dropout=False, do_concat=True):
                dec = deconv2d(tf.nn.relu(x), [self.batch_size, output_width,
                                               output_width, output_filters], scope="g_d%d_deconv" % layer)
                if layer != 8:
                    # IMPORTANT: normalization for last layer
                    # Very important, otherwise GAN is unstable
                    # Trying conditional instance normalization to
                    # overcome the fact that batch normalization offers
                    # different train/test statistics
                    if inst_norm:
                        dec = conditional_instance_norm(dec, ids, self.embedding_num, scope="g_d%d_inst_norm" % layer)
                    else:
                        dec = batch_normalization(dec, is_training, scope="g_d%d_bn" % layer)
                if dropout:
                    dec = tf.nn.dropout(dec, 0.5)
                if do_concat:
                    dec = tf.concat([dec, enc_layer], 3)
                return dec

            d1 = decode_layer(encoded, s128, self.generator_dim * 8, layer=1, enc_layer=encoding_layers["e7"],
                              dropout=True)
            d2 = decode_layer(d1, s64, self.generator_dim * 8, layer=2, enc_layer=encoding_layers["e6"], dropout=True)
            d3 = decode_layer(d2, s32, self.generator_dim * 8, layer=3, enc_layer=encoding_layers["e5"], dropout=True)
            d4 = decode_layer(d3, s16, self.generator_dim * 8, layer=4, enc_layer=encoding_layers["e4"])
            d5 = decode_layer(d4, s8, self.generator_dim * 4, layer=5, enc_layer=encoding_layers["e3"])
            d6 = decode_layer(d5, s4, self.generator_dim * 2, layer=6, enc_layer=encoding_layers["e2"])
            d7 = decode_layer(d6, s2, self.generator_dim, layer=7, enc_layer=encoding_layers["e1"])
            d8 = decode_layer(d7, s, self.output_filters, layer=8, enc_layer=None, do_concat=False)

            output = tf.nn.tanh(d8)  # scale to (-1, 1)
            return output

    def generator(self, images, embeddings, embedding_ids, inst_norm, is_training, reuse=False):
        e8, enc_layers = self.encoder(images, is_training=is_training, reuse=reuse)
        local_embeddings = tf.nn.embedding_lookup(embeddings, ids=embedding_ids)
        local_embeddings = tf.reshape(local_embeddings, [self.batch_size, 1, 1, self.embedding_dim])
        embedded = tf.concat([e8, local_embeddings], 3)
        output = self.decoder(embedded, enc_layers, embedding_ids, inst_norm, is_training=is_training, reuse=reuse)
        return output, e8

    def discriminator(self, image, is_training, reuse=False):
        with tf.variable_scope("discriminator"):
            if reuse:
                tf.get_variable_scope().reuse_variables()
            h0 = lrelu(conv2d(image, self.discriminator_dim, scope="d_h0_conv"))
            h1 = lrelu(batch_normalization(conv2d(h0, self.discriminator_dim * 2, scope="d_h1_conv"),
                                  is_training, scope="d_bn_1"))
            h2 = lrelu(batch_normalization(conv2d(h1, self.discriminator_dim * 4, scope="d_h2_conv"),
                                  is_training, scope="d_bn_2"))
            h3 = lrelu(batch_normalization(conv2d(h2, self.discriminator_dim * 8, sh=1, sw=1, scope="d_h3_conv"),
                                  is_training, scope="d_bn_3"))
            # real or fake binary loss
            fc1 = fc(tf.reshape(h3, [self.batch_size, -1]), 1, scope="d_fc1")
            # category loss
            fc2 = fc(tf.reshape(h3, [self.batch_size, -1]), self.embedding_num, scope="d_fc2")

            return tf.nn.sigmoid(fc1), fc1, fc2

    def build_model(self, is_training=True, inst_norm=False, no_target_source=False):
        real_data = tf.placeholder(tf.float32,
                                   [self.batch_size, self.input_width, self.input_width,
                                    self.input_filters + self.output_filters],
                                   name='real_A_and_B_images')
        embedding_ids = tf.placeholder(tf.int64, shape=None, name="embedding_ids")
        no_target_data = tf.placeholder(tf.float32,
                                        [self.batch_size, self.input_width, self.input_width,
                                         self.input_filters + self.output_filters],
                                        name='no_target_A_and_B_images')
        no_target_ids = tf.placeholder(tf.int64, shape=None, name="no_target_embedding_ids")

        # target images
        real_B = real_data[:, :, :, :self.input_filters]
        # source images
        real_A = real_data[:, :, :, self.input_filters:self.input_filters + self.output_filters]

        embedding = init_embedding(self.embedding_num, self.embedding_dim)
        fake_B, encoded_real_A = self.generator(real_A, embedding, embedding_ids, is_training=is_training,
                                                inst_norm=inst_norm)
        real_AB = tf.concat([real_A, real_B], 3)
        fake_AB = tf.concat([real_A, fake_B], 3)

        # Note it is not possible to set reuse flag back to False
        # initialize all variables before setting reuse to True
        real_D, real_D_logits, real_category_logits = self.discriminator(real_AB, is_training=is_training, reuse=False)
        fake_D, fake_D_logits, fake_category_logits = self.discriminator(fake_AB, is_training=is_training, reuse=True)

        # encoding constant loss
        # this loss assume that generated imaged and real image
        # should reside in the same space and close to each other
        encoded_fake_B = self.encoder(fake_B, is_training, reuse=True)[0]
        const_loss = (tf.reduce_mean(tf.square(encoded_real_A - encoded_fake_B))) * self.Lconst_penalty

        # category loss
        true_labels = tf.reshape(tf.one_hot(indices=embedding_ids, depth=self.embedding_num),
                                 shape=[self.batch_size, self.embedding_num])
        real_category_loss = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=real_category_logits,
                                                                                    labels=true_labels))
        fake_category_loss = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=fake_category_logits,
                                                                                    labels=true_labels))
        category_loss = self.Lcategory_penalty * (real_category_loss + fake_category_loss)

        # binary real/fake loss
        d_loss_real = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=real_D_logits,
                                                                             labels=tf.ones_like(real_D)))
        d_loss_fake = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=fake_D_logits,
                                                                             labels=tf.zeros_like(fake_D)))
        # L1 loss between real and generated images
        l1_loss = self.L1_penalty * tf.reduce_mean(tf.abs(fake_B - real_B))
        # total variation loss
        width = self.output_width
        tv_loss = (tf.nn.l2_loss(fake_B[:, 1:, :, :] - fake_B[:, :width - 1, :, :]) / width
                   + tf.nn.l2_loss(fake_B[:, :, 1:, :] - fake_B[:, :, :width - 1, :]) / width) * self.Ltv_penalty

        # maximize the chance generator fool the discriminator
        cheat_loss = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=fake_D_logits,
                                                                            labels=tf.ones_like(fake_D)))

        d_loss = d_loss_real + d_loss_fake + category_loss / 2.0
        g_loss = cheat_loss + l1_loss + self.Lcategory_penalty * fake_category_loss + const_loss + tv_loss

        if no_target_source:
            # no_target source are examples that don't have the corresponding target images
            # however, except L1 loss, we can compute category loss, binary loss and constant losses with those examples
            # it is useful when discriminator get saturated and d_loss drops to near zero
            # those data could be used as additional source of losses to break the saturation
            no_target_A = no_target_data[:, :, :, self.input_filters:self.input_filters + self.output_filters]
            no_target_B, encoded_no_target_A = self.generator(no_target_A, embedding, no_target_ids,
                                                              is_training=is_training,
                                                              inst_norm=inst_norm, reuse=True)
            no_target_labels = tf.reshape(tf.one_hot(indices=no_target_ids, depth=self.embedding_num),
                                          shape=[self.batch_size, self.embedding_num])
            no_target_AB = tf.concat([no_target_A, no_target_B], 3)
            no_target_D, no_target_D_logits, no_target_category_logits = self.discriminator(no_target_AB,
                                                                                            is_training=is_training,
                                                                                            reuse=True)
            encoded_no_target_B = self.encoder(no_target_B, is_training, reuse=True)[0]
            no_target_const_loss = tf.reduce_mean(
                tf.square(encoded_no_target_A - encoded_no_target_B)) * self.Lconst_penalty
            no_target_category_loss = tf.reduce_mean(
                tf.nn.sigmoid_cross_entropy_with_logits(logits=no_target_category_logits,
                                                        labels=no_target_labels)) * self.Lcategory_penalty

            d_loss_no_target = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=no_target_D_logits,
                                                                                      labels=tf.zeros_like(
                                                                                          no_target_D)))
            cheat_loss += tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=no_target_D_logits,
                                                                                 labels=tf.ones_like(no_target_D)))
            d_loss = d_loss_real + d_loss_fake + d_loss_no_target + (category_loss + no_target_category_loss) / 3.0
            g_loss = cheat_loss / 2.0 + l1_loss + \
                     (self.Lcategory_penalty * fake_category_loss + no_target_category_loss) / 2.0 + \
                     (const_loss + no_target_const_loss) / 2.0 + tv_loss

        d_loss_real_summary = tf.summary.scalar("d_loss_real", d_loss_real)
        d_loss_fake_summary = tf.summary.scalar("d_loss_fake", d_loss_fake)
        category_loss_summary = tf.summary.scalar("category_loss", category_loss)
        cheat_loss_summary = tf.summary.scalar("cheat_loss", cheat_loss)
        l1_loss_summary = tf.summary.scalar("l1_loss", l1_loss)
        fake_category_loss_summary = tf.summary.scalar("fake_category_loss", fake_category_loss)
        const_loss_summary = tf.summary.scalar("const_loss", const_loss)
        d_loss_summary = tf.summary.scalar("d_loss", d_loss)
        g_loss_summary = tf.summary.scalar("g_loss", g_loss)
        tv_loss_summary = tf.summary.scalar("tv_loss", tv_loss)

        d_merged_summary = tf.summary.merge([d_loss_real_summary, d_loss_fake_summary,
                                             category_loss_summary, d_loss_summary])
        g_merged_summary = tf.summary.merge([cheat_loss_summary, l1_loss_summary,
                                             fake_category_loss_summary,
                                             const_loss_summary,
                                             g_loss_summary, tv_loss_summary])

        # expose useful nodes in the graph as handles globally
        input_handle = InputHandle(real_data=real_data,
                                   embedding_ids=embedding_ids,
                                   no_target_data=no_target_data,
                                   no_target_ids=no_target_ids)

        loss_handle = LossHandle(d_loss=d_loss,
                                 g_loss=g_loss,
                                 const_loss=const_loss,
                                 l1_loss=l1_loss,
                                 category_loss=category_loss,
                                 cheat_loss=cheat_loss,
                                 tv_loss=tv_loss)

        eval_handle = EvalHandle(encoder=encoded_real_A,
                                 generator=fake_B,
                                 target=real_B,
                                 source=real_A,
                                 embedding=embedding)

        summary_handle = SummaryHandle(d_merged=d_merged_summary,
                                       g_merged=g_merged_summary)

        # those operations will be shared, so we need
        # to make them visible globally
        setattr(self, "input_handle", input_handle)
        setattr(self, "loss_handle", loss_handle)
        setattr(self, "eval_handle", eval_handle)
        setattr(self, "summary_handle", summary_handle)

    def register_session(self, sess):
        self.sess = sess

    def retrieve_trainable_vars(self, freeze_encoder=False):
        t_vars = tf.trainable_variables()

        d_vars = [var for var in t_vars if 'd_' in var.name]
        g_vars = [var for var in t_vars if 'g_' in var.name]

        if freeze_encoder:
            # exclude encoder weights
            print("freeze encoder weights")
            g_vars = [var for var in g_vars if not ("g_e" in var.name)]

        return g_vars, d_vars

    def retrieve_generator_vars(self):
        all_vars = tf.global_variables()
        generate_vars = [var for var in all_vars if 'embedding' in var.name or "g_" in var.name]
        return generate_vars

    def retrieve_handles(self):
        input_handle = getattr(self, "input_handle")
        loss_handle = getattr(self, "loss_handle")
        eval_handle = getattr(self, "eval_handle")
        summary_handle = getattr(self, "summary_handle")

        return input_handle, loss_handle, eval_handle, summary_handle

    def get_model_id_and_dir(self):
        model_id = "experiment_%d_batch_%d" % (self.experiment_id, self.batch_size)
        model_dir = os.path.join(self.checkpoint_dir, model_id)
        return model_id, model_dir

    def checkpoint(self, saver, step):
        model_name = "unet.model"
        model_id, model_dir = self.get_model_id_and_dir()

        if not os.path.exists(model_dir):
            os.makedirs(model_dir)

        saver.save(self.sess, os.path.join(model_dir, model_name), global_step=step)

    def restore_model(self, saver, model_dir):

        ckpt = tf.train.get_checkpoint_state(model_dir)

        if ckpt:
            saver.restore(self.sess, ckpt.model_checkpoint_path)
            print("restored model %s" % model_dir)
        else:
            print("fail to restore model %s" % model_dir)

    def generate_fake_samples(self, input_images, embedding_ids):
        input_handle, loss_handle, eval_handle, summary_handle = self.retrieve_handles()
        fake_images, real_images, \
        d_loss, g_loss, l1_loss = self.sess.run([eval_handle.generator,
                                                 eval_handle.target,
                                                 loss_handle.d_loss,
                                                 loss_handle.g_loss,
                                                 loss_handle.l1_loss],
                                                feed_dict={
                                                    input_handle.real_data: input_images,
                                                    input_handle.embedding_ids: embedding_ids,
                                                    input_handle.no_target_data: input_images,
                                                    input_handle.no_target_ids: embedding_ids
                                                })
        return fake_images, real_images, d_loss, g_loss, l1_loss

    def validate_model(self, val_iter, epoch, step):
        labels, codes, images = next(val_iter)
        fake_imgs, real_imgs, d_loss, g_loss, l1_loss = self.generate_fake_samples(images, labels)
        print("Sample: d_loss: %.5f, g_loss: %.5f, l1_loss: %.5f" % (d_loss, g_loss, l1_loss))

        merged_fake_images = merge(scale_back(fake_imgs), [self.batch_size, 1])
        merged_real_images = merge(scale_back(real_imgs), [self.batch_size, 1])
        merged_pair = np.concatenate([merged_real_images, merged_fake_images], axis=1)

        model_id, _ = self.get_model_id_and_dir()

        model_sample_dir = os.path.join(self.sample_dir, model_id)
        if not os.path.exists(model_sample_dir):
            os.makedirs(model_sample_dir)

        sample_img_path = os.path.join(model_sample_dir, "sample_%02d_%04d.png" % (epoch, step))
        misc.imsave(sample_img_path, merged_pair)

    def export_generator(self, save_dir, model_dir, model_name="gen_model"):
        saver = tf.train.Saver()
        self.restore_model(saver, model_dir)

        gen_saver = tf.train.Saver(var_list=self.retrieve_generator_vars())
        gen_saver.save(self.sess, os.path.join(save_dir, model_name), global_step=0)

    def infer(self, source_obj, embedding_ids, model_dir, save_dir, progress_file):
        source_provider = InjectDataProvider(source_obj, None)

        with open(progress_file, 'a') as f:
            f.write("Start")

        if isinstance(embedding_ids, int) or len(embedding_ids) == 1:
            embedding_id = embedding_ids if isinstance(embedding_ids, int) else embedding_ids[0]
            source_iter = source_provider.get_single_embedding_iter(self.batch_size, embedding_id)
        else:
            source_iter = source_provider.get_random_embedding_iter(self.batch_size, embedding_ids)

        tf.global_variables_initializer().run()
        saver = tf.train.Saver(var_list=self.retrieve_generator_vars())
        self.restore_model(saver, model_dir)

        def save_imgs(imgs, count):
            p = os.path.join(save_dir, "inferred_%04d.png" % count)
            save_concat_images(imgs, img_path=p)
#            print("generated images saved at %s" % p)

        def save_sample(imgs, code):
            p = os.path.join(save_dir, "inferred_%s.png" % code)
            save_concat_images(imgs, img_path=p)
#            print("generated images saved at %s" % p)

        count = 0
        batch_buffer = list()
        for labels, codes, source_imgs in source_iter:
            fake_imgs = self.generate_fake_samples(source_imgs, labels)[0]
            for i in range(len(fake_imgs)):
                # Denormalize image
                gray_img = np.uint8(fake_imgs[i][:,:,0]*127.5+127.5)
                pil_img = Image.fromarray(gray_img, 'L')
                # Apply bilateralFilter
                cv_img = np.array(pil_img)
                cv_img = bilateralFilter(cv_img, 5, 10, 10)
                pil_img = Image.fromarray(cv_img)
                # Increase contrast
                enhancer = ImageEnhance.Contrast(pil_img)
                en_img = enhancer.enhance(1.5)
                # Normalize image
                fake_imgs[i][:,:,0] = Image.fromarray(np.array(en_img)/127.5 - 1.)
#                save_sample(fake_imgs[i], codes[i])
            merged_fake_images = merge(scale_back(fake_imgs), [self.batch_size, 1])
            batch_buffer.append(merged_fake_images)
            if len(batch_buffer) == 1:
                save_sample(batch_buffer, codes[0])
                batch_buffer = list()
            count += 1
        if batch_buffer:
            # last batch
            save_imgs(batch_buffer, count)
        with open(progress_file, 'a') as f:
            f.write("Done")


    def infer_compare(self, source_obj, embedding_ids, model_dir, save_dir, show_ssim):
        if isinstance(embedding_ids, int) or len(embedding_ids) == 1:
            source_provider = InjectDataProvider(source_obj, [embedding_ids])
            embedding_id = embedding_ids if isinstance(embedding_ids, int) else embedding_ids[0]
            source_iter = source_provider.get_single_embedding_iter(self.batch_size, embedding_id)
            source_length = len(source_provider.data.examples)
        else:
            source_provider = InjectDataProvider(source_obj, embedding_ids)
            source_iter = source_provider.get_random_embedding_iter(self.batch_size, embedding_ids)

        tf.global_variables_initializer().run()
        saver = tf.train.Saver(var_list=self.retrieve_generator_vars())
        self.restore_model(saver, model_dir)

        def save_imgs(imgs, count):
            p = os.path.join(save_dir, "inferred_%04d.png" % count)
            save_concat_images(imgs, img_path=p)
            print("generated images saved at %s" % p)

        def mse(img1, img2):
            return np.linalg.norm(img1 - img2)

        count = 0
        ssim_sum = 0
        sample_num = 0
        S = []
        batch_buffer = list()
        for labels, codes, source_imgs in source_iter:
            fake_imgs, real_imgs, d_loss, g_loss, l1_loss = self.generate_fake_samples(source_imgs, labels)
            if show_ssim:
                S = np.empty([self.batch_size, 128, 128, 1])
            for i in range(len(fake_imgs)):
                # Denormalize image
                gray_img = np.uint8(fake_imgs[i][:,:,0]*127.5+127.5)
                pil_img = Image.fromarray(gray_img, 'L')
                # Apply bilateralFilter
                cv_img = np.array(pil_img)
                cv_img = bilateralFilter(cv_img, 5, 10, 10)
                pil_img = Image.fromarray(cv_img)
                # Increase contrast
                enhancer = ImageEnhance.Contrast(pil_img)
                en_img = enhancer.enhance(1.5)
                # Normalize image
                fake_imgs[i][:,:,0] = Image.fromarray(np.array(en_img)/127.5 - 1.)

                ssim_i = ssim(fake_imgs[i], real_imgs[i], multichannel=True)
                if show_ssim:
                    mean, Ssim = ssim(fake_imgs[i], real_imgs[i], full=True, multichannel=True)
                    S[i]=Ssim
                ssim_sum = ssim_sum + ssim_i
                sample_num = sample_num + 1
#                print("[%d] SSIM: %.5f" % (sample_num, ssim_i))
            merged_fake_images = merge(scale_back(fake_imgs), [self.batch_size, 1])
            merged_real_images = merge(scale_back(real_imgs), [self.batch_size, 1])
            merged_pair = np.concatenate([merged_real_images, merged_fake_images], axis=1)
            if show_ssim:
                merged_pair = np.concatenate([merged_pair, merge(scale_back(S), [self.batch_size, 1])], axis=1)
            batch_buffer.append(merged_pair)
            if len(batch_buffer) == 10:
                save_imgs(batch_buffer, count)
                batch_buffer = list()
            count += 1
        if batch_buffer:
            # last batch
            save_imgs(batch_buffer, count)
        print("Average SSIM: %.5f" % (ssim_sum/sample_num))

    def interpolate(self, source_obj, between, model_dir, save_dir, steps):
        tf.global_variables_initializer().run()
        saver = tf.train.Saver(var_list=self.retrieve_generator_vars())
        self.restore_model(saver, model_dir)
        # new interpolated dimension
        new_x_dim = steps + 1
        alphas = np.linspace(0.0, 1.0, new_x_dim)

        def _interpolate_tensor(_tensor):
            """
            Compute the interpolated tensor here
            """

            x = _tensor[between[0]]
            y = _tensor[between[1]]

            interpolated = list()
            for alpha in alphas:
                interpolated.append(x * (1. - alpha) + alpha * y)

            interpolated = np.asarray(interpolated, dtype=np.float32)
            return interpolated

        def filter_embedding_vars(var):
            var_name = var.name
            if var_name.find("embedding") != -1:
                return True
            if var_name.find("inst_norm/shift") != -1 or var_name.find("inst_norm/scale") != -1:
                return True
            return False

        embedding_vars = filter(filter_embedding_vars, tf.trainable_variables())
        # here comes the hack, we overwrite the original tensor
        # with interpolated ones. Note, the shape might differ

        # this is to restore the embedding at the end
        embedding_snapshot = list()
        for e_var in embedding_vars:
            val = e_var.eval(session=self.sess)
            embedding_snapshot.append((e_var, val))
            t = _interpolate_tensor(val)
            op = tf.assign(e_var, t, validate_shape=False)
            print("overwrite %s tensor" % e_var.name, "old_shape ->", e_var.get_shape(), "new shape ->", t.shape)
            self.sess.run(op)

        source_provider = InjectDataProvider(source_obj, None)
        input_handle, _, eval_handle, _ = self.retrieve_handles()
        for step_idx in range(len(alphas)):
            alpha = alphas[step_idx]
            print("interpolate %d -> %.4f + %d -> %.4f" % (between[0], 1. - alpha, between[1], alpha))
            source_iter = source_provider.get_single_embedding_iter(self.batch_size, 0)
            batch_buffer = list()
            count = 0
            for _, codes, source_imgs in source_iter:
                count += 1
                labels = [step_idx] * self.batch_size
                generated, = self.sess.run([eval_handle.generator],
                                           feed_dict={
                                               input_handle.real_data: source_imgs,
                                               input_handle.embedding_ids: labels
                                           })
                merged_fake_images = merge(scale_back(generated), [self.batch_size, 1])
                batch_buffer.append(merged_fake_images)
            if len(batch_buffer):
                save_concat_images(batch_buffer,
                                   os.path.join(save_dir, "frame_%02d_%02d_step_%02d.png" % (
                                       between[0], between[1], step_idx)))
        # restore the embedding variables
        print("restore embedding values")
        for var, val in embedding_snapshot:
            op = tf.assign(var, val, validate_shape=False)
            self.sess.run(op)

    def train(self, lr=0.0002, epoch=100, schedule=10, resume=True, flip_labels=False,
              freeze_encoder=False, fine_tune=None, sample_steps=50, checkpoint_steps=500, no_val=False):
        g_vars, d_vars = self.retrieve_trainable_vars(freeze_encoder=freeze_encoder)
        input_handle, loss_handle, _, summary_handle = self.retrieve_handles()

        if not self.sess:
            raise Exception("no session registered")

        with open(self.progress_file, 'a') as f:
            f.write("Start")

        learning_rate = tf.placeholder(tf.float32, name="learning_rate")
        d_optimizer = tf.train.AdamOptimizer(learning_rate, beta1=0.5).minimize(loss_handle.d_loss, var_list=d_vars)
        g_optimizer = tf.train.AdamOptimizer(learning_rate, beta1=0.5).minimize(loss_handle.g_loss, var_list=g_vars)
        tf.global_variables_initializer().run()
        real_data = input_handle.real_data
        embedding_ids = input_handle.embedding_ids
        no_target_data = input_handle.no_target_data
        no_target_ids = input_handle.no_target_ids

        # filter by one type of labels
        data_provider = TrainDataProvider(self.data_dir, filter_by=fine_tune, no_val=no_val)
        total_batches = data_provider.compute_total_batch_num(self.batch_size)
        val_batch_iter = 0
        if not no_val:
            val_batch_iter = data_provider.get_val_iter(self.batch_size)

        saver = tf.train.Saver(max_to_keep=3)
        summary_writer = tf.summary.FileWriter(self.log_dir, self.sess.graph)

        if resume:
            _, model_dir = self.get_model_id_and_dir()
            self.restore_model(saver, model_dir)

        current_lr = lr
        counter = 0
        start_time = time.time()

        for ei in range(epoch):
            train_batch_iter = data_provider.get_train_iter(self.batch_size)

            if (ei + 1) % schedule == 0:
                update_lr = current_lr / 2.0
                # minimum learning rate guarantee
                update_lr = max(update_lr, 0.0002)
                print("decay learning rate from %.5f to %.5f" % (current_lr, update_lr))
                current_lr = update_lr

            for bid, batch in enumerate(train_batch_iter):
                counter += 1
                labels, codes, batch_images = batch
                shuffled_ids = labels[:]
                if flip_labels:
                    np.random.shuffle(shuffled_ids)
                # Optimize D
                _, batch_d_loss, d_summary = self.sess.run([d_optimizer, loss_handle.d_loss,
                                                            summary_handle.d_merged],
                                                           feed_dict={
                                                               real_data: batch_images,
                                                               embedding_ids: labels,
                                                               learning_rate: current_lr,
                                                               no_target_data: batch_images,
                                                               no_target_ids: shuffled_ids
                                                           })
                # Optimize G
                _, batch_g_loss = self.sess.run([g_optimizer, loss_handle.g_loss],
                                                feed_dict={
                                                    real_data: batch_images,
                                                    embedding_ids: labels,
                                                    learning_rate: current_lr,
                                                    no_target_data: batch_images,
                                                    no_target_ids: shuffled_ids
                                                })
                # magic move to Optimize G again
                # according to https://github.com/carpedm20/DCGAN-tensorflow
                # collect all the losses along the way
                _, batch_g_loss, category_loss, cheat_loss, \
                const_loss, l1_loss, tv_loss, g_summary = self.sess.run([g_optimizer,
                                                                         loss_handle.g_loss,
                                                                         loss_handle.category_loss,
                                                                         loss_handle.cheat_loss,
                                                                         loss_handle.const_loss,
                                                                         loss_handle.l1_loss,
                                                                         loss_handle.tv_loss,
                                                                         summary_handle.g_merged],
                                                                        feed_dict={
                                                                            real_data: batch_images,
                                                                            embedding_ids: labels,
                                                                            learning_rate: current_lr,
                                                                            no_target_data: batch_images,
                                                                            no_target_ids: shuffled_ids
                                                                        })
                passed = time.time() - start_time
                log_format = "Epoch: [%2d], [%4d/%4d] time: %4.4f, d_loss: %.5f, g_loss: %.5f, " + \
                             "category_loss: %.5f, cheat_loss: %.5f, const_loss: %.5f, l1_loss: %.5f, tv_loss: %.5f"
#                print(log_format % (ei, bid, total_batches, passed, batch_d_loss, batch_g_loss,
#                                    category_loss, cheat_loss, const_loss, l1_loss, tv_loss))
                summary_writer.add_summary(d_summary, counter)
                summary_writer.add_summary(g_summary, counter)

                if (not no_val) and counter % sample_steps == 0:
                    # sample the current model states with val data
                    self.validate_model(val_batch_iter, ei, counter)

                if counter % checkpoint_steps == 0:
                    print(log_format % (ei, bid, total_batches, passed, batch_d_loss, batch_g_loss,
                                    category_loss, cheat_loss, const_loss, l1_loss, tv_loss))
                    print("Checkpoint: save checkpoint step %d" % counter)
                    self.checkpoint(saver, counter)
        # save the last checkpoint
        print("Checkpoint: last checkpoint step %d" % counter)
        self.checkpoint(saver, counter)
        with open(self.progress_file, 'a') as f:
            f.write("Done")


In [91]:

import easydict

args = easydict.EasyDict({
 
        "experiment_dir": 'C:/Users/1-16/python/neural-fonts-master/experiment',
 
        "experiment_id": 0,

 
        "image_size": 128,
 
        "L1_penalty": 100,
    
        "Lconst_penalty": 15,
    
        "Ltv_penalty": 0.0,
 
        "Lcategory_penalty": 1.0,
    
        "embedding_num": 40,
 
        "embedding_dim": 128,
        "batch_size": 16,
        "lr": 0.001,
    
        "schedule": 10,
        "resume": 1,
        "freeze_encoder": 0,
        "fine_tune": None,
        "inst_norm": 0,
        "sample_steps": 10,
        "checkpoint_steps": 500,
        "flip_labels": None,
        "no_val": None,
        "epoch": 30
 
    
 
})

In [118]:
# -*- coding: utf-8 -*-
from __future__ import print_function
from __future__ import absolute_import

import tensorflow as tf
import argparse

# import tensorflow.compat.v1 as tf
# tf.disable_v1_behavior() 

# %tensorflow_version 1.x
# import tensorflow as tf

import tensorflow.compat.v1 as tf



import easydict

args = easydict.EasyDict({
 
        "experiment_dir": 'C:/Users/1-16/python/neural-fonts-master/experiment',
 
        "experiment_id": 0,

 
        "image_size": 128,
 
        "L1_penalty": 100,
    
        "Lconst_penalty": 15,
    
        "Ltv_penalty": 0.0,
 
        "Lcategory_penalty": 1.0,
    
        "embedding_num": 40,
 
        "embedding_dim": 128,
        "batch_size": 16,
        "lr": 0.001,
    
        "schedule": 10,
        "resume": 1,
        "freeze_encoder": 0,
        "fine_tune": None,
        "inst_norm": 0,
        "sample_steps": 10,
        "checkpoint_steps": 500,
        "flip_labels": 1,
        "no_val": None,
        "epoch": 30
 
    
 
})


tf.reset_default_graph()


def main(_):
    config = tf.ConfigProto()
    config.gpu_options.allow_growth = True

    with tf.Session(config=config) as sess:
        model = UNet(args.experiment_dir, batch_size=args.batch_size, experiment_id=args.experiment_id,
                     input_width=args.image_size, output_width=args.image_size, embedding_num=args.embedding_num,
                     embedding_dim=args.embedding_dim, L1_penalty=args.L1_penalty, Lconst_penalty=args.Lconst_penalty,
                     Ltv_penalty=args.Ltv_penalty, Lcategory_penalty=args.Lcategory_penalty)
        model.register_session(sess)
        if args.flip_labels:
            model.build_model(is_training=True, inst_norm=args.inst_norm, no_target_source=True)
        else:
            model.build_model(is_training=True, inst_norm=args.inst_norm)
        fine_tune_list = None
        if args.fine_tune:
            ids = args.fine_tune.split(",")
            fine_tune_list = set([int(i) for i in ids])
        model.train(lr=args.lr, epoch=args.epoch, resume=args.resume,
                    schedule=args.schedule, freeze_encoder=args.freeze_encoder, fine_tune=fine_tune_list,
                    sample_steps=args.sample_steps, checkpoint_steps=args.checkpoint_steps,
                    flip_labels=args.flip_labels, no_val=args.no_val)


if __name__ == '__main__':
    tf.app.run()

processed 1000 examples
processed 2000 examples
processed 3000 examples
processed 4000 examples
processed 5000 examples
processed 6000 examples
processed 7000 examples
processed 8000 examples
processed 9000 examples
processed 10000 examples
unpickled total 10456 examples
processed 1000 examples
unpickled total 1166 examples
train examples -> 10456, val examples -> 1166


IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



IndexError: tuple index out of range